# Steam Tables

Two packages:
- IAPWS (license = GNU, which limits uses) https://pypi.org/project/iapws/
- Chemicals (license = MIT, which is a more open license and therefore preferred) https://chemicals.readthedocs.io/

The chemicals package uses SI units...  

so pressure is in Pascals (Pa)
- 1.e6 Pa = 1 Mega Pascal (MPa)
- 1.e5 Pa = 1 bar

and temperature is Kelvin
- Degrees celsius = Kelvin - 273.15

Ensure that pressure is absolute not gauge. 

Data used in this notebooks comes from https://github.com/ICWallis/T21-Tutorial-WellTestAnalysis

In [ ]:
from chemicals import iapws
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# What is the saturation temperature for 30 bara?

P_Pa = 30.e5
Tsat_C = iapws.iapws95_Tsat(P_Pa) - 273.15 # convert from K to C
print(round(Tsat_C,2))

In [ ]:
# What is the saturation pressure for 100 degC?

Temp_C = 100
T_K = Temp_C + 273.15 # convert from C to K
Psat_bar = iapws.iapws95_Psat(T_K) / 1.e5 # convert from Pa to bar

print(round(Psat_bar,2))

In [ ]:
# For a given temperature - pressure pair, calculate fluid properties enthalpy and density

T_K = 233.85 + 273.15
P_Pa = 30.e5

properties = iapws.iapws95_properties(T=T_K, P=P_Pa)

print(properties)

H_kJpkg = properties[3] * 1.e-3 # convert J/kg to kJ/kg
density = properties[0] # kg/m3

print(round(H_kJpkg,2), round(density,2))

The fluid properties function returns a tuple and the method above slices out the values we want. 

Refer to the docs for the tuple definition: 
https://chemicals.readthedocs.io/chemicals.iapws.html#iapws-95-properties

## Calculate the BPD for a static temperature profile

In [ ]:
data = pd.read_csv('3-Static-PT.csv')

In [ ]:
data.info()

In [ ]:
data['pres_Pa'] = (data.pres_barg + 1) * 1.e5

In [ ]:
data.head()

In [ ]:
# assuming pure water, calculate the BPD curve

well_Tsat_C = []

for val in data.pres_Pa.values:
    Tsat_C = iapws.iapws95_Tsat(val) - 273.15
    well_Tsat_C.append(Tsat_C)

data['Tsat_degC'] = well_Tsat_C

In [ ]:
data.columns

In [ ]:
fig, ax = plt.subplots(1,1,figsize = (4,4))

ax.plot(
    data.temp_degC.values,
    data.depth_m.values,
    label = 'Measured temp'
)

ax.plot(
    data.Tsat_degC.values,
    data.depth_m.values,
    label = 'Saturation temp'
)

ax.set_ylim(1000,0)
ax.set_ylabel('Depth [m]')
ax.set_xlabel('Temp [$^\circ$C]') # Latex used to format unit
ax.legend()

## What if it's not pure water?

Work in progress, contributors welcome...

https://github.com/Geothermal-Fluids/bpd_curve